In [233]:
import pandas as pd
import gurobipy as gp
import math
import pickle as pkl
from datetime import datetime, timedelta

In [234]:
product_type = "ambient"
num_time_periods = 7

In [235]:
article_data = pd.read_csv("./data/article.csv")
article_data = article_data[(article_data['TEMPERATURE_ZONE'] == product_type)]


In [236]:

if(product_type=="frozen"):
    warehouse_volume = 30
if(product_type=="chilled"):
    warehouse_volume = 300
if(product_type=="ambient"):
    warehouse_volume = 700
buffer_cost = 25
#max_order_exceed_multiplier = 1.5
max_order_exceed_penalty = 0

In [237]:
def createParameterMatrix(data, columns):
    parameters = []
    for column in columns:
        parameters.append(data[column].to_list())
    parameters = list(map(list, zip(*parameters)))
    return parameters

In [238]:
articles = article_data['ARTICLE_ID'].to_list()

parameters = createParameterMatrix(
    article_data,
    [
        'TEMPERATURE_ZONE',
        'VOLUME_M3_PER_CU',
        'MEAN_SHELF_LIFE',
        'CU_PER_TU',
        'ORDERING_COST_FIXED',
        'ORDERING_COST_PER_TU',
        'CLEARING_COST_PER_CU',
        'MINIMUM_ORDER_QUANTITY_TU',
        'MAXIMUM_ORDER_QUANTITY_TU'
    ]
)
parameters_dict = dict(zip(articles, parameters))

In [239]:
items, category, volume_per_cu, shelf_life, cu_per_tu, ordering_cost_fixed, ordering_cost_per_tu, clearing_cost_per_cu, minimum_order_quantity_tu, maximum_order_quantity_tu = gp.multidict(parameters_dict)

In [240]:
forecast_data = pd.read_csv('./data/sales_'+str(num_time_periods)+'.csv')
forecast_data = forecast_data[forecast_data['ARTICLE_ID'].isin(articles)]

In [241]:
# Create a new dataframe with all dates
all_dates_df = pd.DataFrame({'DATE': pd.date_range(start=min(forecast_data['DATE']), end=max(forecast_data['DATE']), freq='D')}).astype(str)

# Group the original dataframe by item
grouped = forecast_data.groupby('ARTICLE_ID')

# Initialize an empty list to store the new dataframes
new_dfs = []

# Loop over each group
for item, group_df in grouped:
    
    group_df['DATE'] = pd.to_datetime(group_df['DATE']).astype(str)

    # Merge the group dataframe with the all_dates dataframe
    merged_df = pd.merge(all_dates_df, group_df, on='DATE', how='outer')
    merged_df['ARTICLE_ID'] = item
    
    # Fill in missing values
    merged_df['PICKING_QUANTITY_CU'] = merged_df['PICKING_QUANTITY_CU'].fillna(0)
    
    # Sort by date and append to the list
    new_dfs.append(merged_df.sort_values('DATE'))
    
# Concatenate all new dataframes into a single dataframe
forecast_data = pd.concat(new_dfs)
time_periods = forecast_data['DATE'].unique()

time_indexes = [*range(len(time_periods))]
date_to_index = {time_periods[i]:[*range(len(time_periods))][i] for i in time_indexes}
index_to_date = {[*range(len(time_periods))][i]:time_periods[i] for i in time_indexes}

# demand = forecast_data.groupby('DATE').apply(lambda x: dict(zip(x['ARTICLE_ID'], x['PICKING_QUANTITY_CU']))).to_dict()
# demand = dict((date_to_index[key],value) for (key,value) in demand.items())

demand = forecast_data.groupby('ARTICLE_ID').apply(lambda x: dict(zip(x['DATE'], x['PICKING_QUANTITY_CU']))).to_dict()
for item in demand.keys():
    demand[item] = dict((date_to_index[key], value) for (key, value) in demand[item].items())

In [242]:
cost_dict = {
    'fixed':0,
    'per_tu':0,
    'clearance':0,
    'buffer':0,
    'max_order_penalty':0,
    'total':0
}

In [243]:
def getCost(orders, order_date, cu_per_tu_item, cum_demand, disposal_cost):
        cost = 0
        if (orders[order_date]* cu_per_tu_item) - cum_demand > 0:
            cost = ((orders[order_date] * cu_per_tu_item) - cum_demand) * disposal_cost
        return cost   

def getMaxOrderPenalty(orders, order_date, cu_per_tu_item, variable_cost, maximum_order_quantity_item):
    max_order = 0
    if orders[order_date] > maximum_order_quantity_item: 
        #max_order = ((orders[order_date] * cu_per_tu_item) - maximum_order_quantity_item) * max_order_exceed_multiplier * variable_cost
        max_order = ((orders[order_date]) - maximum_order_quantity_item)  + max_order_exceed_penalty 
    return max_order 

In [244]:
def silverMeal(item):
    
    item_demand = list(demand[item].values())
    fixed_cost = ordering_cost_fixed[item]
    variable_cost = ordering_cost_per_tu[item]
    cu_per_tu_item = cu_per_tu[item]
    disposal_cost = clearing_cost_per_cu[item]
    maximum_order_quantity_tu_item = maximum_order_quantity_tu[item]
    minimum_order_quantity_tu_item = minimum_order_quantity_tu[item]
    life = shelf_life[item]
    cost = 0
    tot_disposal = 0
    tot_fixed = 0
    tot_ordering = 0
    tot_max_order = 0
    penalty = 0

    orders = {}
    order_date = 0 
    run_complete = False
    obj_val = 0
    while(not run_complete):
        if (order_date %7 == 6):
            order_date = order_date + 1 
            continue   
        
        min_cost = float('inf')
        cum_demand = 0
        
        for t in range(order_date, len(item_demand)):
            if(t%7==6 ):
                continue
            cum_demand += item_demand[t]
            if(t==order_date+life):
                orders[order_date] = math.ceil(cum_demand/cu_per_tu_item)
                if (orders[order_date] < minimum_order_quantity_tu_item):
                        orders[order_date] = minimum_order_quantity_tu_item
                #cost+= getCost(orders, order_date, cu_per_tu_item, cum_demand, disposal_cost) + getMaxOrderPenalty(orders, order_date, cu_per_tu_item, variable_cost, maximum_order_quantity_tu_item)
                tot_disposal += getCost(orders, order_date, cu_per_tu_item, cum_demand, disposal_cost) 
                tot_ordering += math.ceil(orders[order_date]) * variable_cost
                tot_fixed += fixed_cost
                tot_max_order +=  getMaxOrderPenalty(orders, order_date, cu_per_tu_item, variable_cost, maximum_order_quantity_tu_item)
                order_date = t + 1
                break

            cost = fixed_cost + variable_cost * math.ceil(cum_demand/cu_per_tu_item) 
            if(math.ceil(cum_demand/cu_per_tu_item) > maximum_order_quantity_tu_item):
                penalty = (math.ceil(cum_demand/cu_per_tu_item) - maximum_order_quantity_tu_item)  + max_order_exceed_penalty
                #penalty = (math.ceil(cum_demand/cu_per_tu_item) - maximum_order_quantity_tu_item) * max_order_exceed_multiplier  
            avg_cost = (cost + penalty) / (t-order_date+1)
            
            if(avg_cost <= min_cost):
                min_cost = avg_cost
                if(t==len(item_demand)-1):
                    
                    orders[order_date] = math.ceil(cum_demand/cu_per_tu_item)
                    if (orders[order_date] < minimum_order_quantity_tu_item):
                                            orders[order_date] = minimum_order_quantity_tu_item

                    #cost+= getCost(orders, order_date, cu_per_tu_item, cum_demand, disposal_cost) + getMaxOrderPenalty(orders, order_date, cu_per_tu_item,variable_cost, maximum_order_quantity_tu_item)
                    tot_disposal += getCost(orders, order_date, cu_per_tu_item, cum_demand, disposal_cost)
                    tot_ordering += math.ceil(orders[order_date]) * variable_cost
                    tot_fixed += fixed_cost   
                    tot_max_order +=  getMaxOrderPenalty(orders, order_date, cu_per_tu_item, variable_cost, maximum_order_quantity_tu_item)
                 
                    run_complete = True
                    break

            else:
                orders[order_date] = math.ceil((cum_demand- item_demand[t])/cu_per_tu_item)
                if (orders[order_date] < minimum_order_quantity_tu_item):
                        orders[order_date] = minimum_order_quantity_tu_item
                #cost+= getCost(orders, order_date, cu_per_tu_item, cum_demand, disposal_cost) + getMaxOrderPenalty(orders, order_date, cu_per_tu_item, variable_cost, maximum_order_quantity_tu_item)
                tot_disposal += getCost(orders, order_date, cu_per_tu_item, cum_demand- item_demand[t], disposal_cost)
                tot_ordering += math.ceil(orders[order_date]) * variable_cost
                tot_fixed += fixed_cost
                tot_max_order +=  getMaxOrderPenalty(orders, order_date, cu_per_tu_item, variable_cost, maximum_order_quantity_tu_item)

                order_date = t
                break
       
        obj_val += cost
        if(order_date > len(item_demand)-1):
            run_complete = True
            break
    tot_obj_val = tot_disposal +  tot_ordering + tot_fixed + tot_max_order     
    return (orders, tot_obj_val, tot_disposal, tot_ordering, tot_fixed, tot_max_order)

obj_val = 0 
vol = 0
tot_disposal = 0
tot_fixed = 0
tot_ordering = 0
tot_max_order = 0

schedule = {}
for item in demand.keys():
    orders, val, disposal, ordering, fixed, max_order_p = silverMeal(item) 
    obj_val += val
    tot_disposal += disposal
    tot_fixed += fixed
    tot_ordering += ordering
    tot_max_order += max_order_p
    for t in orders.keys():
        if orders[t] == 0:
            continue
        if item not in schedule.keys():
            schedule[item] = {}
        if t not in schedule[item].keys():
            schedule[item][t] = 0
        schedule[item][t] += orders[t]
        
    
    
#print(silverMeal('00ee8964'))
print(obj_val)
print('Ordering',tot_ordering)
print('Fixing ', tot_fixed)
print('Clearance ', tot_disposal)
print('Max Order ', tot_max_order)


949149.4500000032
Ordering 764934.3999999985
Fixing  106309.69999999995
Clearance  69210.00000000019
Max Order  8695.35


In [245]:
buf_cost = 0
vol = 0
time_indexes
volumes = {}

for time in range(len(time_indexes)):
    for item in schedule:
        if time in schedule[item].keys():
            vol += schedule[item][time] * cu_per_tu[item] * volume_per_cu[item]

    if time > 0:
        for item in demand.keys():
            if time in demand[item].keys():
                vol -= demand[item][time] * volume_per_cu[item]

    volumes[time] = vol
    if vol > warehouse_volume:

        buf_cost += (vol - warehouse_volume) * buffer_cost

buf_cost
obj_val = obj_val + buf_cost

In [246]:
cost_dict['total'] = obj_val
cost_dict['buffer'] = buf_cost
cost_dict['clearance'] = tot_disposal
cost_dict['fixed'] = tot_fixed
cost_dict['per_tu'] = tot_ordering
cost_dict['max_order_penalty'] = tot_max_order
cost_dict

{'fixed': 106309.69999999995,
 'per_tu': 764934.3999999985,
 'clearance': 69210.00000000019,
 'buffer': 0,
 'max_order_penalty': 8695.35,
 'total': 949149.4500000032}

In [247]:
obj_val

949149.4500000032

In [248]:
schedule

{'00027afc': {0: 3, 4: 2},
 '00084396': {0: 2},
 '0025253f': {0: 4, 2: 6},
 '00299fa6': {0: 10, 3: 10},
 '002b8d7d': {0: 2},
 '0034e982': {0: 3, 2: 4},
 '00415c43': {0: 14, 4: 10},
 '0057e9b7': {0: 5, 2: 19},
 '0059a703': {0: 12, 4: 5},
 '0062b624': {0: 5},
 '00718852': {0: 12},
 '007d0c15': {0: 1, 4: 1},
 '009aa26d': {0: 8},
 '00d09a16': {0: 2, 4: 1},
 '00f7bc48': {0: 1, 4: 1},
 '00f95978': {0: 18},
 '010001ce': {0: 2, 5: 1},
 '01039c78': {0: 5, 4: 4},
 '010a9e81': {0: 1},
 '011f544e': {0: 1, 4: 1},
 '01527ae0': {0: 1},
 '01581e6a': {0: 46},
 '015bf73f': {0: 2, 3: 2},
 '016d54fd': {0: 2, 5: 2},
 '0173b075': {0: 3, 4: 3},
 '018a81b7': {0: 4, 5: 1},
 '018afad9': {0: 4, 3: 5},
 '019b9022': {0: 1},
 '01c1f8cb': {0: 2},
 '01c4e30e': {0: 8, 3: 8},
 '01c5bf11': {0: 1, 4: 1},
 '01cac4fc': {0: 1},
 '01cb9c42': {0: 4, 2: 8, 4: 9},
 '01ea7fcd': {0: 8},
 '01ed569a': {0: 20},
 '01f22aa0': {0: 1, 3: 1},
 '01f433bd': {0: 1, 3: 2},
 '02031c33': {0: 94},
 '0220bc95': {0: 1},
 '022168b8': {0: 2, 3: 1, 

In [249]:
def neighbourhoodOperator1(schedule, obj_val):
    max_improvement = math.inf
    cost = obj_val
    modified_schedule = {}
    day_wise_orders = {}

    buffer_penalty_tot = 0
    max_order_penalty_tot = 0
    fixed_ordering_cost_tot = 0 
    schedule_imp = schedule 
    count = 0
    iterator = 0
    MAX_ITER = 100
    MAX_REPEAT = 3
    prev_max_improvement = 0

    while count < MAX_REPEAT and iterator < MAX_ITER:
        for item in schedule_imp:
            buffer_penalty = 0
            max_order_penalty = 0 
            cost_improvement = 0
            modified_orders = {}
            # Check if the item has a second order date
            if len(schedule_imp[item]) > 1:
                order_days = list(schedule_imp[item].keys())
                for i in range (0,len(order_days)-1, 2):
                    day1 = i
                    day2 = i + 1
                    if order_days[day2] - order_days[day1] <= shelf_life[item]:
                        tot_orders = schedule_imp[item][order_days[day1]]+ schedule_imp[item][order_days[day2]]
                        vol = tot_orders * cu_per_tu[item] * volume_per_cu[item]
                        if vol + volumes[day1] > warehouse_volume:
                            buffer_penalty += buffer_cost * (vol + volumes[day1] - warehouse_volume)
                        if tot_orders > maximum_order_quantity_tu[item]:
                            max_order_penalty += ( tot_orders - maximum_order_quantity_tu[item])  + max_order_exceed_penalty
                        delta = - ordering_cost_fixed[item] + buffer_penalty + max_order_penalty 
                        print(delta)
                        if delta < 0:
                            fixed_ordering_cost_tot += ordering_cost_fixed[item] 
                            buffer_penalty_tot += buffer_penalty
                            max_order_penalty_tot += max_order_penalty
                            cost += delta
                            modified_orders[order_days[day1]] = tot_orders
                            volumes[order_days[day1]] += vol
                        else:
                            modified_orders[order_days[day1]] = schedule_imp[item][order_days[day1]]
                            modified_orders[order_days[day2]] = schedule_imp[item][order_days[day2]]
                modified_schedule[item] = modified_orders
            
            else:
                if len(schedule[item]) == 1 : 
                    modified_schedule[item] = schedule_imp[item]
                else:
                    continue


            if cost!=0 and cost < max_improvement:
                max_improvement = cost
        
        print("Buffer penalty paid", buffer_penalty_tot)
        print("max order penalty paid", max_order_penalty_tot)
        print("fixed cost reduced by", fixed_ordering_cost_tot)
        schedule_imp = modified_schedule
        obj_val = max_improvement
        iterator +=1

        print(max_improvement)

        if(max_improvement==prev_max_improvement):
            count+=1
        prev_max_improvement = max_improvement
    
    return schedule_imp, obj_val, buffer_penalty_tot, max_order_penalty_tot, fixed_ordering_cost_tot

In [250]:
schedule, obj_val, buffer_cost_delta, max_order_penalty_delta, fixed_cost_delta = neighbourhoodOperator1(schedule, obj_val)
cost_dict['buffer'] += buffer_cost_delta
cost_dict['max_order_penalty'] += max_order_penalty_delta
cost_dict['fixed'] -= fixed_cost_delta
cost_dict['total'] = obj_val
cost_dict

-10.3
-11.6
77.5
-14.4
-5.7
-9.1
-13.1
-14.9
-11.0
-17.8
-10.0
-9.5
-20.5
-6.7
-18.2
-13.2
-18.0
-11.3
-10.0
-20.3
-13.5
-19.9
-18.4
-13.4
-8.1
-12.5
-9.4
-16.7
-14.7
-20.1
-14.2
-5.2
-14.0
-10.6
46.0
-17.0
-9.1
-9.4
-20.5
-17.0
17.2
-19.2
-7.6
-14.6
-15.2
-12.6
-17.3
-13.1
92.0
13.499999999999998
-11.7
-14.2
-6.3
-19.5
-9.8
-10.4
-13.5
-8.0
-17.4
-17.6
-14.0
-14.1
-13.4
-14.7
-8.6
-13.5
-13.5
-11.9
-18.5
-12.7
-16.7
-13.1
-20.0
-16.7
-16.3
-16.6
-18.4
-11.4
-15.3
-18.7
-13.3
-17.7
-17.5
-15.0
-5.3
-15.7
-4.4
-14.0
-15.3
-9.9
-15.5
-19.1
-12.4
-12.7
-20.5
-20.0
-14.4
-16.3
-15.3
-9.9
-18.1
-17.0
-17.0
69.1
-7.7
-12.4
-20.4
-8.5
-15.3
-16.1
-11.1
-14.7
-16.0
-9.9
-9.4
-9.5
-5.2
-11.5
-13.8
-19.4
-19.7
-4.6
-11.0
-15.9
-12.8
-8.2
-19.7
-11.6
-8.7
-14.6
-12.1
-16.0
-10.3
-16.4
-15.5
-7.3
-16.4
-6.8
-12.2
-11.9
20.5
-16.3
-20.0
-8.4
-18.2
-15.1
-14.9
-9.0
-10.3
-7.7
7.2
-12.0
-11.1
-4.8
-11.7
-19.3
-4.6
-16.4
-14.5
-18.6
610.7
-20.2
-9.0
-19.9
-19.9
-4.0
143.2
-14.9
-14.8
-12.5
-14.1
-11.1

{'fixed': 84390.19999999994,
 'per_tu': 764934.3999999985,
 'clearance': 69210.00000000019,
 'buffer': 264.71280760029003,
 'max_order_penalty': 8714.85,
 'total': 927514.162807605}

In [251]:
def transposeScheduleToDayFirst(schedule):
    result_dict = {}
    for day, articles in schedule.items():
        for article, quantity in articles.items():
            if article in result_dict:
                result_dict[article][day] = quantity
            else:
                result_dict[article] = {day: quantity}
    return result_dict

def transposeScheduleToItemFirst(schedule):
    result_dict = {}
    for article, days in schedule.items():
        for day, quantity in days.items():
            if day in result_dict:
                result_dict[day][article] = quantity
            else:
                result_dict[day] = {article: quantity}
    return result_dict

In [252]:
new_clearance_cost = 0
ordering_cost_per_tu_saved = 0

# Add clearance cost
# If I ever have to throw away equal to or more than a TU's worth, then I can just reduce order
def removeExpired(inv,life,item):
    global ordering_cost_per_tu_saved
    global new_clearance_cost
    
    tu_saved = 0
    # if inventory has items from days which are now too old
    if(len(inv)>life):
        amount_cleared = inv[0]

        tu_saved = amount_cleared//cu_per_tu[item]
        # ordering cost reduced here
        ordering_cost_per_tu_saved += tu_saved * ordering_cost_per_tu[item]

        amount_inevitably_cleared = amount_cleared%cu_per_tu[item]
        # clearance cost recalculated here
        new_clearance_cost += amount_inevitably_cleared*clearing_cost_per_cu[item]
        
        # remove days which are now too old
        inv = inv[-life:]
    
    return inv, tu_saved

def addDay(inv,ordered):
    inv.append(ordered)
    return inv

# how much of the current demand can be satisfied from items in inventory
def satisfyFromInventory(inv, dem):
    for day in range(len(inv)):
        if(dem==0):
            break
        if(inv[day] <= dem):
            dem -= inv[day]
            inv[day] = 0
        else:
            inv[day] -= dem
            dem = 0
    return inv

def updateInventory(inv,item,t,ordered):
    life = shelf_life[item]
    if(item in demand[t].keys()):
        dem = demand[t][item]
    else:
        dem = 0
    # first the orders of the new day are added to the inventory
    # and then the total inventory is used to satisfy the demand
    # This would mean that the demand cannot go unsatisfied from
    # the items in the inventory
    inv = addDay(inv, ordered)
    inv, tu_saved = removeExpired(inv,life,item)
    inv = satisfyFromInventory(inv, dem)
    return inv, tu_saved

def calculateMaxOrderPenalty(schedule):
    max_order_penalty = 0
    for day in schedule.keys():
        for item in schedule[day].keys():
            if(schedule[day][item] > maximum_order_quantity_tu[item]):
                max_order_penalty += (schedule[day][item] - maximum_order_quantity_tu[item])  + max_order_exceed_penalty
    return max_order_penalty

def endClearance(schedule, item, inv):
    cu_surplus = sum(inv)
    tu_surplus = cu_surplus // cu_per_tu[item]
    cu_disposed = cu_surplus % cu_per_tu[item]
    for day in range(len(schedule)-1,-1,-1):
        if(day in schedule.keys()):
            if(tu_surplus <= 0):
                break
            if(schedule[day] >= tu_surplus and schedule[day] - tu_surplus >= minimum_order_quantity_tu[item]):
                schedule[day] -= tu_surplus
                # save on per TU cost
                break
            else:
                tu_surplus -= schedule[day]
                schedule[day] = 0
                # save on per TU and fixed TU cost
    cu_disposed += tu_surplus * cu_per_tu[item]
    clearance_cost = cu_disposed * clearing_cost_per_cu[item]
    return schedule, clearance_cost

# def calculateEndClearance(inv,item):
#     cu_available = sum(inv)
#     tu_saved = cu_available//cu_per_tu[item]
#     cu_disposed = cu_available%cu_per_tu[item]
#     clearance_cost = cu_disposed * clearing_cost_per_cu[item]
#     return tu_saved, clearance_cost

# def removeFinalOrders(schedule, tu_saved, item):
#     for day in range(len(schedule)-1,-1,-1):
#         if(day in schedule.keys()):
#             if(tu_saved <= 0):
#                 break
#             if(schedule[day] >= tu_saved and schedule[day] - tu_saved >= minimum_order_quantity_tu[item]):
#                 schedule[day] -= tu_saved
#                 break
#             else:
#                 tu_saved -= schedule[day]
#                 schedule[day] = 0
#     return schedule

def neighbourhoodOperator2(schedule):
    # dictionary of format {'item':['q left on earliest possible','q left on earliest possible + 1',...,'q left today']}
    global new_clearance_cost
    global ordering_cost_per_tu_saved
    inventory = {}

    for t in time_indexes:
        for item in articles:

            if item not in inventory.keys():
                inventory[item] = []

            # if there was no order for an item in a particular day, then the order was 0
            if(t in schedule.keys() and item in schedule[t].keys()):
                ordered = schedule[t][item] * cu_per_tu[item]
            else:
                ordered = 0

            # update the inventory according to what has been ordered today for item
            inventory[item],tu_saved = updateInventory(inventory[item], item, t, ordered)
            
            # Removing excess TUs from the schedule. On day t, we remove the excess from day t-shelf life
            if(t-shelf_life[item] in schedule.keys() and item in schedule[t-shelf_life[item]].keys()):
                if(t>=shelf_life[item]):
                    if(schedule[t-shelf_life[item]][item] - tu_saved < minimum_order_quantity_tu[item]):
                        diff = minimum_order_quantity_tu[item] - (schedule[t-shelf_life[item]][item] - tu_saved)
                        schedule[t-shelf_life[item]][item] = minimum_order_quantity_tu[item]
                        ordering_cost_per_tu_saved -= diff * ordering_cost_per_tu[item]
                    else:
                        schedule[t-shelf_life[item]][item] -= tu_saved



    schedule = transposeScheduleToItemFirst(schedule)
    for item in schedule.keys():
        schedule[item], end_clearance_cost = endClearance(schedule[item], item, inventory[item])
        new_clearance_cost += end_clearance_cost
    schedule = transposeScheduleToDayFirst(schedule)
                
    print("Clearance cost: ", new_clearance_cost)
    print("Ordering cost per TU saved: ", ordering_cost_per_tu_saved)

    new_max_order_penalty = calculateMaxOrderPenalty(schedule)

    return schedule, new_clearance_cost, ordering_cost_per_tu_saved, new_max_order_penalty

In [253]:
def transposeDemand(demand):
    result_dict = {}
    for item, days in demand.items():
        for day, quantity in days.items():
            if day in result_dict:
                result_dict[day][item] = quantity
            else:
                result_dict[day] = {item:quantity}
    return result_dict
demand = transposeDemand(demand)

In [254]:
schedule = transposeScheduleToDayFirst(schedule)
schedule, new_clearance_cost, per_tu_delta, new_max_order_penalty = neighbourhoodOperator2(schedule)
cost_dict['clearance'] = new_clearance_cost
cost_dict['max_order_penalty'] = new_max_order_penalty
cost_dict['per_tu'] -= per_tu_delta
cost_dict['total'] = cost_dict['fixed'] + cost_dict['per_tu'] + cost_dict['buffer'] + cost_dict['clearance'] + cost_dict['max_order_penalty']

Clearance cost:  30989.29999999998
Ordering cost per TU saved:  4496.099999999999


In [255]:
cost_dict

{'fixed': 84390.19999999994,
 'per_tu': 760438.2999999985,
 'clearance': 30989.29999999998,
 'buffer': 264.71280760029003,
 'max_order_penalty': 8162.250000000001,
 'total': 884244.7628075987}